In [1]:
import os
import xml.etree.ElementTree as et
from helpers import *

PATH = '/home/user/Documentos/HHHA-2018-01895_Dr.JCA' #carpeta con las imagenes!

**Archivos xml (anotaciones) disponibles**

In [2]:
xmls = filenames(PATH, 'xml')
xmls

['HHHA-2018-01895 A1 20-60.xml',
 'HHHA-2018-01895 B1 20-60.xml',
 'HHHA-2018-01895 B2 00-40.xml']

In [3]:
tree = et.parse(os.path.join(PATH, xmls[1]))
root = tree.getroot()
print(tree)

In [4]:
print(root)

<Element 'Annotations' at 0x7f1b718d4040>


### Estructura general xml

Para acceder a los datos relevantes

In [5]:
for a in root:
    Id, name, color = a.get('Id'), a.get('Name'), a.get('LineColor')
    print(a, f'{Id = }, {name = }, {color = }')
    
    for region in a[1][1:]:
        print(region[1], f' vertex: {len(region[1])}')
    print('')

<Element 'Annotation' at 0x7f1b718d40e0> Id = '1', name = 'Layer 1', color = '255'
<Element 'Vertices' at 0x7f1b718d4310>  vertex: 679
<Element 'Vertices' at 0x7f1b719358f0>  vertex: 3092
<Element 'Vertices' at 0x7f1b7172a3e0>  vertex: 1726
<Element 'Vertices' at 0x7f1b71658270>  vertex: 1632

<Element 'Annotation' at 0x7f1b7155c310> Id = '2', name = '', color = '65408'
<Element 'Vertices' at 0x7f1b7155c6d0>  vertex: 93
<Element 'Vertices' at 0x7f1b7155e4d0>  vertex: 649
<Element 'Vertices' at 0x7f1b713c7150>  vertex: 427
<Element 'Vertices' at 0x7f1b71407830>  vertex: 187
<Element 'Vertices' at 0x7f1b7142f3d0>  vertex: 102
<Element 'Vertices' at 0x7f1b7144d4e0>  vertex: 161
<Element 'Vertices' at 0x7f1b71468860>  vertex: 608
<Element 'Vertices' at 0x7f1b712d0810>  vertex: 987
<Element 'Vertices' at 0x7f1b71353e70>  vertex: 225
<Element 'Vertices' at 0x7f1b71398630>  vertex: 1057
<Element 'Vertices' at 0x7f1b712412b0>  vertex: 164
<Element 'Vertices' at 0x7f1b7125c720>  vertex: 249

<E

In [6]:
for vertex in root[4][1][1][1]:
    print(vertex.attrib)

{'X': '987', 'Y': '1193', 'Z': '0'}
{'X': '1152', 'Y': '1152', 'Z': '0'}
{'X': '1275', 'Y': '1069', 'Z': '0'}
{'X': '1481', 'Y': '1028', 'Z': '0'}
{'X': '1522', 'Y': '987', 'Z': '0'}
{'X': '1563', 'Y': '905', 'Z': '0'}
{'X': '1687', 'Y': '864', 'Z': '0'}
{'X': '1769', 'Y': '823', 'Z': '0'}
{'X': '1892', 'Y': '699', 'Z': '0'}
{'X': '2180', 'Y': '452', 'Z': '0'}
{'X': '2304', 'Y': '411', 'Z': '0'}
{'X': '2509', 'Y': '288', 'Z': '0'}
{'X': '2592', 'Y': '164', 'Z': '0'}
{'X': '2674', 'Y': '82', 'Z': '0'}
{'X': '2756', 'Y': '82', 'Z': '0'}
{'X': '2797', 'Y': '41', 'Z': '0'}
{'X': '2838', 'Y': '41', 'Z': '0'}
{'X': '2880', 'Y': '41', 'Z': '0'}
{'X': '2921', 'Y': '41', 'Z': '0'}
{'X': '2962', 'Y': '41', 'Z': '0'}
{'X': '3044', 'Y': '41', 'Z': '0'}
{'X': '3085', 'Y': '41', 'Z': '0'}
{'X': '3168', 'Y': '0', 'Z': '0'}
{'X': '3209', 'Y': '0', 'Z': '0'}
{'X': '3291', 'Y': '-41', 'Z': '0'}
{'X': '3332', 'Y': '-41', 'Z': '0'}
{'X': '3497', 'Y': '-123', 'Z': '0'}
{'X': '3579', 'Y': '-123', 'Z': '0'}


Se tiene la siguiente relación

**LineColor___Color___Significado___**

255---------Rojo------Tejido Con Cáncer

65408------Verde-----Tejido Sin Cáncer

65535-----Amarillo---

16711808---Morado ----

16776960 ---Celeste---Delimita donde están las anotaciones


---

## Lectura poligonos por color

In [7]:
def get_vert(fn, color):
    root = et.parse(fn).getroot()
    v = []
    for a in root:
        if a.get('LineColor') == str(color):
            for region in a[1][1:]:
                v.append( [( int(float(c.get('X'))),  int(float(c.get('Y'))) )  for c in region[1]] )
    return v

#retorna lista con listas(poligonos) | celeste siempre es sólo 1 anotación
p=get_vert(os.path.join(PATH, xmls[2]), 16776960) 
p[0]

[(38528, -1324), (71518, -1324), (71518, 23719), (38528, 23719)]

In [8]:
d = get_all_vert(PATH, 16776960)
d.keys()

dict_keys(['HHHA-2018-01895 B1 20-60', 'HHHA-2018-01895 B2 00-40', 'HHHA-2018-01895 A1 20-60'])

Nos sirve conocer las coordenadas que limitan el poligono celeste, pues no requerimos leer lo que esté fuera de él.

In [9]:
def boundbox_pol(pol):
    Max_x = max(point[0] for point in pol)
    Min_x = min(point[0] for point in pol)
    Max_y = max(point[1] for point in pol)
    Min_y = min(point[1] for point in pol)
    print(f'{Min_x = }, {Max_x = }, {Min_y = }, {Max_y = }')
    return Min_x, Max_x, Min_y, Max_y
    
boundbox_pol(d['HHHA-2018-01895 B1 20-60'][0])

Min_x = 494, Max_x = 32745, Min_y = -247, Max_y = 16290


(494, 32745, -247, 16290)